In [2]:
!pip install streamlit pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [3]:
%%writefile requirements.txt
streamlit
pandas
numpy
scikit-learn
seaborn
matplotlib
pyngrok
plotly
prophet

Writing requirements.txt


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
from prophet import Prophet
import plotly.graph_objs as go

# Sample data
data = {
    'state': ['Maharashtra', 'Karnataka', 'Punjab', 'Uttar Pradesh', 'Tamil Nadu',
              'Andhra Pradesh', 'West Bengal', 'Rajasthan', 'Bihar', 'Haryana',
              'Madhya Pradesh', 'Gujarat', 'Odisha', 'Chhattisgarh', 'Jharkhand',
              'Kerala', 'Assam', 'Telangana', 'Goa', 'Delhi'],
    'district': ['Nashik', 'Bangalore', 'Ludhiana', 'Meerut', 'Salem',
                 'Guntur', 'Hooghly', 'Jaipur', 'Patna', 'Hisar',
                 'Indore', 'Ahmedabad', 'Cuttack', 'Raipur', 'Ranchi',
                 'Ernakulam', 'Guwahati', 'Hyderabad', 'Panaji', 'New Delhi'],
    'market': ['Nashik Market', 'Bangalore Market', 'Ludhiana Market', 'Meerut Market', 'Salem Market',
               'Guntur Market', 'Hooghly Market', 'Jaipur Market', 'Patna Market', 'Hisar Market',
               'Indore Market', 'Ahmedabad Market', 'Cuttack Market', 'Raipur Market', 'Ranchi Market',
               'Ernakulam Market', 'Guwahati Market', 'Hyderabad Market', 'Panaji Market', 'Azadpur Market'],
    'crop': ['Onion', 'Tomato', 'Wheat', 'Potato', 'Turmeric',
             'Chilli', 'Rice', 'Mustard', 'Maize', 'Bajra',
             'Soybean', 'Cotton', 'Groundnut', 'Paddy', 'Pulses',
             'Pepper', 'Tea', 'Sorghum', 'Cashew', 'Fruit Basket'],
    'modal_price': [1300, 1000, 2000, 700, 8500,
                    10500, 1600, 5000, 1250, 1800,
                    3900, 5500, 4700, 1500, 5200,
                    33000, 180, 1900, 11000, 1500]
}

# Generate synthetic time series for each market (past 60 days)
import numpy as np
from datetime import datetime, timedelta

historical_data = []
start_date = datetime.today() - timedelta(days=60)

for i in range(len(data['market'])):
    market = data['market'][i]
    state = data['state'][i]
    district = data['district'][i]
    crop = data['crop'][i]
    base_price = data['modal_price'][i]
    for j in range(60):
        date = start_date + timedelta(days=j)
        # Simulate prices with some noise
        price = base_price + np.random.normal(0, base_price * 0.05)
        historical_data.append({
            'date': date.strftime('%Y-%m-%d'),
            'state': state,
            'district': district,
            'market': market,
            'crop': crop,
            'price': round(price, 2)
        })

df = pd.DataFrame(historical_data)

# Streamlit UI
st.title("Agri Market Price Viewer with Prophet Forecasting")

# State selection
state = st.selectbox("Select State", df['state'].unique())
filtered_df = df[df['state'] == state]

district = st.selectbox("Select District", filtered_df['district'].unique())
filtered_df = filtered_df[filtered_df['district'] == district]

market = st.selectbox("Select Market", filtered_df['market'].unique())
filtered_df = filtered_df[filtered_df['market'] == market]

# Display historical prices
st.subheader(f"Historical Prices for {filtered_df.iloc[0]['crop']} at {market}")
st.line_chart(filtered_df.set_index('date')['price'])

# Prophet forecasting
st.subheader("Forecast Prices (Next 30 Days)")

# Prepare data for Prophet
prophet_df = filtered_df[['date', 'price']].rename(columns={'date': 'ds', 'price': 'y'})
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

# Fit Prophet model
model = Prophet(daily_seasonality=True)
model.fit(prophet_df)

# Forecast next 30 days
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Plot forecast using Plotly
fig = go.Figure()

# Historical prices
fig.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['y'], mode='lines', name='Historical'))

# Forecasted prices
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))

# Confidence intervals
fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_upper'],
    mode='lines',
    name='Upper Confidence',
    line=dict(width=0),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat_lower'],
    mode='lines',
    name='Lower Confidence',
    fill='tonexty',
    line=dict(width=0),
    fillcolor='rgba(0,100,80,0.2)',
    showlegend=False
))

fig.update_layout(title=f"Price Forecast for {filtered_df.iloc[0]['crop']} at {market}",
                  xaxis_title='Date', yaxis_title='Price')

st.plotly_chart(fig)



Writing app.py


In [6]:

from pyngrok import ngrok
import threading
import time

# Kill existing tunnels
ngrok.kill()

# Run Streamlit app in a thread
def run_streamlit():
    !streamlit run app.py --server.enableCORS false

thread = threading.Thread(target=run_streamlit)
thread.start()

# Wait for Streamlit to start
time.sleep(5)

# Authenticate ngrok - This should be done before connecting
!ngrok authtoken 2vtaAI5WYfoOFhCbalbuzii23gb_UFpyWciHpU49nsg3rLqR  # Replace with your actual authtoken

# Get the public URL from ngrok
# The 'addr' argument is used to specify the port
public_url = ngrok.connect(addr=8501)
print(f"Streamlit app is live at: {public_url}")

2025-06-01 17:17:31.391 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.145.221.138:8502

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://88d7-34-145-221-138.ngrok-free.app" -> "http://localhost:8501"
